In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from use_case.baseline import * 
from tests.eval import *

payoff_i = np.random.uniform(-10, 10, (20, 20))
payoff_j = np.transpose(payoff_i).copy()

# Initialize environment
N_ACTIONS = payoff_i.shape[0]
N_AGENTS = 1000
env = BaselineEnvironment(N_AGENTS, payoff_i, payoff_j, total_games = 1)

# Actual Run

In [3]:
from models.model import *
from models.trainer import *


In [4]:
# Configure the network here
parameters = ParameterSettings(
    n_agents = N_AGENTS,
    d_action = N_ACTIONS, 
    d_obs = env.obs_size, 
    d_traits = 1,
    d_beliefs = 1
)
parameters.device = "cuda" if torch.cuda.is_available() else "cpu"

model = Model(parameters)

In [5]:
equilibriua = find_pure_equilibria(payoff_i, payoff_j)

for eq in equilibriua:
    x, y = eq 
    a = (y[0] + y[1]) / 2

    print(x, a)

In [6]:
evaluate_policy(model, env, 10)

Average Return: -0.1680346371399749
Total returns: -1.680346371399749
Action Distribution
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 74  11  83   9  16  60  17  52  32  24  10  16  16 132 125  20  10  59
 127 107]


In [7]:
# Setup the training loop
training_parameters = TrainingParameters(
    actor_training_loops = 10,
    hypernet_training_loops= 5, 
    outer_loops = 200,
    actor_learning_rate= 0.01,
    experience_buffer_size = 3,
)

train_model(model, env, training_parameters)
        

Epoch 0


Hypernet Loop: 100%|██████████| 5/5 [00:03<00:00,  1.26it/s]


Average Return: 2.5579683071271595
Total returns: 25.579683071271596
Action Distribution
[ 0  1  6  7  8  9 10 14]
[272   4 159  36  22 251   1 255]


Actor Training: 100%|██████████| 10/10 [00:02<00:00,  3.34it/s]


Average Return: 2.0487213702583262
Total returns: 20.48721370258326
Action Distribution
[ 0  1  6  7  8  9 14]
[256  39 188  53  28 169 267]
Epoch 1


...

Hypernet Loop: 100%|██████████| 5/5 [00:02<00:00,  2.33it/s]


Average Return: 5.142014508533021
Total returns: 51.420145085330205
Action Distribution
[0 1 6 8 9]
[938  48   9   4   1]


Actor Training: 100%|██████████| 10/10 [00:01<00:00,  6.70it/s]


Average Return: 5.087788956849017
Total returns: 50.87788956849017
Action Distribution
[0 1 6]
[964  28   8]
Epoch 50


KeyboardInterrupt: 

In [ ]:
for i in range(0, payoff_i.shape[0]):
    print(payoff_i[i][i])